### References

* https://github.com/huggingface/transformers/blob/master/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

* https://github.com/huggingface/transformers/blob/c4d4e8bdbd25d9463d41de6398940329c89b7fb6/src/transformers/generation_utils.py#L101

* https://github.com/hiyoung123/SoftMaskedBert/blob/master/train.py

* https://github.com/atulkum/pointer_summarizer/blob/master/training_ptr_gen/model.py

* https://github.com/nyu-dl/bert-gen/blob/master/bert-babble.ipynb


In [1]:
!pip install transformers==4.0.1
!pip install nlpaug

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 387 kB 3.0 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import time
import datetime

import re
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

from tqdm import tqdm
import time
import random

import nlpaug.augmenter.char as nac

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, \
                         AutoConfig, EncoderDecoderModel

from typing import Optional

from transformers import AdamW, get_linear_schedule_with_warmup

from transformers.configuration_utils import PretrainedConfig
from transformers.file_utils import add_start_docstrings, add_start_docstrings_to_model_forward, replace_return_docstrings
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import logging
from transformers.models.encoder_decoder.configuration_encoder_decoder import EncoderDecoderConfig
from transformers.models.auto.modeling_auto import AutoModel, AutoModelForCausalLM, AutoConfig

In [3]:
seed_val = 66
MIN_LEN = 20
MAX_LEN = 100
batch_size = 12
EPOCHS = 30
early_stopping_rounds = 5
lr = 5e-5

encoder_path = 'bert-base-uncased'
decoder_path = 'claudelkros/bert-base-french' #'bert-base-german-cased'

teacher_forcing=True

aug = nac.OcrAug()

In [4]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [5]:
torch.cuda.empty_cache()

In [6]:
# !cat ../input/denoising/NMT-data/commoncrawl.fr-en.en | wc -l
# !cat ../input/denoising/NMT-data/commoncrawl.fr-en.fr | wc -l


In [7]:
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        nmt_data =  pd.read_csv(os.path.join(dirname, filename))

/kaggle/input/nmt-eng-french/NMT_denoisedData.csv


In [8]:
print(nmt_data.columns)
print(nmt_data.shape,'\n')

Index(['Unnamed: 0', 'original', 'solution', 'char_augumented',
       'keyborad_augumented', 'random_augumented', 'substitute_augumented',
       'swap_augumented', 'delete_augumented', 'spell_augumented'],
      dtype='object')
(10000, 10) 



In [9]:
train  = nmt_data[['delete_augumented','solution']]

In [10]:
train.columns

Index(['delete_augumented', 'solution'], dtype='object')

In [11]:
train.head(-1)

,delete_augumented,solution
0,* - Main gods are maked with red colr.,* - les marchandises principales sont souligne...
1,* - Man srviss are marked ith red color.,* - les services principales sont soulignes pa...
2,ervies of laguae translation of the. ..,Services de traduction de la documentation...\n
3,An announmet must be commercial charter.,Les messages doivent porter le caractère d’aff...
4,Good and serics avanceme throg the P. O. Box s...,L’avance de ses marchandises et de ses service...
...,...,...
9994,"2: Mari (5 Ich ), why not let other kno?",2: Mario (5 Inch)? Dites ce que vous en pensez!\n
9995,Write a rview for New Super Mari Bros.,Faites part de votre opinion concernant New Su...
9996,the price for New Supe Mario Bos.,Le prix de New Super Mario Bros.\n
9997,there are new imaes or links availab for New S...,De nouvelles images ou de nouveaux liens seron...


In [12]:
train = train.rename(columns={"solution": "solution", "delete_augumented": "original"}, errors="raise")

In [13]:
train.columns

Index(['original', 'solution'], dtype='object')

In [14]:
# train_input = open('../input/denoising/NMT-data/commoncrawl.fr-en.en', 'r').readlines()
# train_output = open('../input/denoising/NMT-data/commoncrawl.fr-en.fr', 'r').readlines()
# train = pd.DataFrame()
# train['original'] = train_input
# train['solution'] = train_output
# train = train.iloc[:10000]
# train.original = train.original.apply(lambda x: aug.augment(x, n=1))

In [15]:
#train = pd.read_csv('../input/denoising/en_de.csv',sep='\t')

In [16]:
train.head()

,original,solution
0,* - Main gods are maked with red colr.,* - les marchandises principales sont souligne...
1,* - Man srviss are marked ith red color.,* - les services principales sont soulignes pa...
2,ervies of laguae translation of the. ..,Services de traduction de la documentation...\n
3,An announmet must be commercial charter.,Les messages doivent porter le caractère d’aff...
4,Good and serics avanceme throg the P. O. Box s...,L’avance de ses marchandises et de ses service...


In [17]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_path)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_path)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [18]:
train['text_len'] = train.original.apply(lambda x: len(encoder_tokenizer.encode(x, max_length=512, add_special_tokens=True)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [19]:
train.text_len.describe()

count    10000.000000
mean        35.847400
std         18.567951
min          8.000000
25%         22.000000
50%         32.000000
75%         46.000000
max        215.000000
Name: text_len, dtype: float64

In [20]:
kf = KFold(n_splits=5)

for train_index, val_index in kf.split(train):
    break

In [21]:
val = train.iloc[val_index]
train = train.iloc[train_index]

In [22]:
trainX = torch.Tensor(np.asarray([encoder_tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                  for i in tqdm(train.original.values)]))
trainy = torch.Tensor(np.asarray([decoder_tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                  for i in tqdm(train.solution.values)]))

valX = torch.Tensor(np.asarray([encoder_tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                for i in tqdm(val.original.values)]))
valy = torch.Tensor(np.asarray([decoder_tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                for i in tqdm(val.solution.values)]))


100%|██████████| 2000/2000 [00:00<00:00, 4311.34it/s]


In [23]:
trainX.shape, trainy.shape, valX.shape, valy.shape

(torch.Size([8000, 100]),
 torch.Size([8000, 100]),
 torch.Size([2000, 100]),
 torch.Size([2000, 100]))

In [24]:
trainX = torch.tensor(trainX, dtype=torch.long)
trainy = torch.tensor(trainy, dtype=torch.long)
valX = torch.tensor(valX, dtype=torch.long)
valy = torch.tensor(valy, dtype=torch.long)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [25]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_path,decoder_path)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441M [00:00<?, ?B/s]

Some weights of the model checkpoint at claudelkros/bert-base-french were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at claudelkros/bert-base-french and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [26]:
model.config.encoder.max_length = MAX_LEN
model.config.decoder.max_length = MAX_LEN

model.config.encoder.min_length = MIN_LEN
model.config.decoder.min_length = MIN_LEN

In [27]:
train_data_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(trainX, trainy), batch_size=batch_size)

val_data_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(valX, valy), batch_size=batch_size)

print ("Train and val loader length {} and {}".format(len(train_data_loader), len(val_data_loader)))

Train and val loader length 667 and 167


In [28]:
print ("Modeling")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("Device: {}".format(device))

model = model.to(device)

params = list(model.named_parameters())

optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_data_loader) * EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Modeling
Device: cuda


In [29]:
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.cuda.empty_cache()

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

bad_epochs = 0
best_val_logits = 0

# For each epoch...
for epoch_i in range(0, EPOCHS):
    
    if bad_epochs < early_stopping_rounds:
        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_data_loader):

            # Progress update every 40 batches.
            if step % 60 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.  Loss: {}'.format(step, len(train_data_loader), elapsed, loss.item()))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            input_ids = batch[0].to(device)
            output_ids = batch[1].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # are given and what flags are set. For our usage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            if teacher_forcing:
                outputs = model(input_ids=input_ids, decoder_input_ids=output_ids, labels=output_ids, return_dict=True)
            else:
                outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=output_ids, return_dict=True)
                
            loss, logits = outputs.loss, outputs.logits

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_data_loader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_loss = 0
        nb_eval_steps = 0

        all_val_logits = []
        
        # Evaluate data for one epoch
        for batch in val_data_loader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            input_ids = batch[0].to(device)
            output_ids = batch[1].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                if teacher_forcing:
                    outputs = model(input_ids=input_ids, decoder_input_ids=output_ids, labels=output_ids, return_dict=True)
                else:
                    outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=output_ids, return_dict=True)
                
                loss, logits = outputs.loss, outputs.logits


            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()

            all_val_logits.extend(logits.argmax(-1))
            
        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(val_data_loader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        if epoch_i >= 1:
            if avg_val_loss < training_stats[-1]['Valid. Loss']:
                #model.save_pretrained('Bert2Bert_denoise')
                torch.save(model.state_dict(), 'gated_Bert2Bert_denoise.bin')
                bad_epochs = 0
                best_val_logits = all_val_logits.copy()
            else:
                bad_epochs += 1

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )
    
    else:
        print ("Early stopping!!")
        break

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 30 ========
Training...
  Batch    60  of    667.    Elapsed: 0:00:27.  Loss: 4.767064094543457
  Batch   120  of    667.    Elapsed: 0:00:53.  Loss: 2.6770272254943848
  Batch   180  of    667.    Elapsed: 0:01:20.  Loss: 1.9633405208587646
  Batch   240  of    667.    Elapsed: 0:01:46.  Loss: 2.256546974182129
  Batch   300  of    667.    Elapsed: 0:02:12.  Loss: 1.7267345190048218
  Batch   360  of    667.    Elapsed: 0:02:38.  Loss: 1.9710339307785034
  Batch   420  of    667.    Elapsed: 0:03:05.  Loss: 1.3134269714355469
  Batch   480  of    667.    Elapsed: 0:03:31.  Loss: 1.7911916971206665
  Batch   540  of    667.    Elapsed: 0:03:57.  Loss: 1.3366572856903076
  Batch   600  of    667.    Elapsed: 0:04:24.  Loss: 1.5542354583740234
  Batch   660  of    667.    Elapsed: 0:04:50.  Loss: 1.682282567024231

  Average training loss: 2.45
  Training epcoh took: 0:04:53

Running Validation...
  Validation Loss: 2.38
  Validation took: 0:00:41

======== Epoch 2 / 

In [30]:
#model.load_state_dict(torch.load('BERT_denoise.bin'))

In [31]:
with open('training_stats.txt','w') as f:
    for l in training_stats:
        f.write(str(l))
        f.write('\n')

In [32]:
for batch in val_data_loader:
    input_ids = batch[0].to(device)
    output_ids = batch[1].to(device)
    break

with torch.no_grad():        
    outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=output_ids, return_dict=True)
    loss, logits = outputs.loss, outputs.logits

In [33]:
original_texts = [decoder_tokenizer.decode(i) for i in output_ids.detach().cpu().numpy()]
predicted_texts = [decoder_tokenizer.decode(i) for i in logits.detach().cpu().numpy().argmax(axis=-1)]

In [34]:
print ("original text ", original_texts[0])
print ("predicted text ", predicted_texts[0])

original text  [CLS] * - les marchandises principales sont soulignes par la couleur rouge. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
predicted text  les les futen de spacious a on auitom les [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [35]:
predicted_texts = []

for i in best_val_logits:
    text = decoder_tokenizer.decode(i).split()
    text = [k for k in text if k not in ['[CLS]','[SEP]','[PAD]']]
    predicted_texts.append(" ".join(text).strip())

In [36]:
val['predicted_text'] = predicted_texts

In [37]:
import numpy as np
import  nltk.translate.bleu_score as bleu

In [38]:
def WRR(text1,text2):
    a = set(text1.lower().split())
    b = set(text2.lower().split())
    
    if (len(a) == 0) and (len(b) == 0):
        return .5
    
    c = a.intersection(b)
    return float(len(c))/(len(a) + len(b) - len(c))

def levenshtein(seq1, seq2):
    seq1 = seq1.lower()
    seq2 = seq2.lower()
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

def CRR(text1, text2):
    try:
        return 1 - float(levenshtein(text1,text2))/max(len(text1),len(text2))
    except:
        return 0

def bleu_score(text1,text2):
    return bleu.sentence_bleu([text1.lower().split()],text2.lower().split())

In [39]:
val['text_len'] = val.original.apply(lambda x: len(encoder_tokenizer.encode(x, max_length=512, add_special_tokens=True)))
val = val[val.text_len < MAX_LEN].reset_index(drop=True)

In [40]:
val['BLEU'] = val.apply(lambda x: bleu_score(x.predicted_text, x.solution), axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [41]:
val.BLEU.describe()

count    1994.000000
mean        0.449842
std         0.212884
min         0.000000
25%         0.347853
50%         0.526640
75%         0.598379
max         0.795271
Name: BLEU, dtype: float64

In [42]:
val.to_csv('nmt_outputs_french_delete_augumented.csv',index=False)